In [4]:
pip install pyOpenBCI

In [7]:
pip install pyqtgraph

In [1]:
from pyOpenBCI import OpenBCICyton

import threading
import time

import sys
sys.path.append('C:/Python37/Lib/site-packages')

from IPython.display import clear_output
%matplotlib qt5
%matplotlib inline


from pyqtgraph.Qt import QtGui, QtCore
import pyqtgraph as pg
import random
import numpy as np
from scipy import signal

import scipy
from scipy.signal import butter, lfilter

def print_raw(sample):
    print(sample.channels_data)

board = OpenBCICyton(port='COM5', daisy=False)

def start_cyton():
    try:
        board.start_stream(print_raw)
    except:
        pass

y = threading.Thread(target=start_cyton)
y.daemon = True
y.start()

time.sleep(.02)
board.disconnect()

Serial established
ID:<159> <Unexpected END_BYTE found <255> instead of <192>
Skipped 10 bytes before start found
[-8388608, -7696286, -7861707, -408264, -5332467, -5826074, -4554524, -6672385]
Closing Serial


In [2]:

SCALE_FACTOR = (4500000)/24/(2**23-1) #From the pyOpenBCI repo
colors = 'rgbycmwr'

# Set up GUI Layout
app = QtGui.QApplication([])
win = pg.GraphicsWindow(title='Python OpenBCI GUI')
ts_plots = [win.addPlot(row=i, col=0, colspan=2, title='Channel %d' % i, labels={'left': 'uV'}) for i in range(1,4)]
fft_plot = win.addPlot(row=1, col=2, rowspan=4, title='FFT Plot', labels={'left': 'uV', 'bottom': 'Hz'})

data = [[0,0,0,0,0,0,0,0]]

# Define OpenBCI callback function
def save_data(sample):
    global data
    data.append([i*SCALE_FACTOR for i in sample.channels_data])

# Define function to update the graphs
def updater():
    global data, plots, colors
    t_data = np.array(data[-1250:]).T #transpose data
    fs = 250 #Hz

    # Notch Filter
    def notch_filter(val, data, fs=250):
        notch_freq_Hz = np.array([float(val)])
        for freq_Hz in np.nditer(notch_freq_Hz):
            bp_stop_Hz = freq_Hz + 3.0 * np.array([-1, 1])
            b, a = signal.butter(3, bp_stop_Hz / (fs / 2.0), 'bandstop')
            fin = data = signal.lfilter(b, a, data)
        return fin

    # Bandpass filter
    def bandpass(start, stop, data, fs = 250):
        bp_Hz = np.array([start, stop])
        b, a = signal.butter(5, bp_Hz / (fs / 2.0), btype='bandpass')
        return signal.lfilter(b, a, data, axis=0)
    
    #     def bandpass(data,low,high, fs = 250, order=5,analog=True):
#         nyq = 0.5 * fs
#         low = low / nyq
#         high = high / nyq
#         b, a = signal.butter(order,[low,high],btype='band')
#         y = signal.lfilter(b,a,data)

    # Applying the filters
    nf_data = [[],[],[],[],[],[],[],[]]
    bp_nf_data = [[],[],[],[],[],[],[],[]]

    for i in range(8):
        nf_data[i] = notch_filter(50, t_data[i])
        bp_nf_data[i] = bandpass(30, 120, nf_data[i])

    # Plot a time series of the raw data
    for j in range(3):
        ts_plots[j].clear()
        ts_plots[j].plot(pen=colors[j]).setData(t_data[j])

    # Get an FFT of the data and plot it
    sp = [[],[],[],[],[],[],[],[]]
    freq = [[],[],[],[],[],[],[],[]]
    
    fft_plot.clear()
    for k in range(3):
        sp[k] = np.absolute(np.fft.fft(bp_nf_data[k]))
        freq[k] = np.fft.fftfreq(bp_nf_data[k].shape[-1], 1.0/fs)
        fft_plot.plot(pen=colors[k]).setData(freq[k], sp[k])

# Define thread function
def start_board():
    board = OpenBCICyton(port='COM5', daisy=False)
    board.start_stream(save_data)
    
# Initialize Board and graphing update
if __name__ == '__main__':
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        x = threading.Thread(target=start_board)
        x.daemon = True
        x.start()

        timer = QtCore.QTimer()
        timer.timeout.connect(updater)
        timer.start(0)


        QtGui.QApplication.instance().exec_()

Serial established
Skipped 10 bytes before start found
Skipped 100 bytes before start found


In [ ]:
board.disconnect()